*AI TRIP PLANNER*

In [ ]:
pip install langchain langchain_core langchain_groq langchain_community langchain langgraph


In [ ]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

DEFINE AGENT

In [ ]:
class PlannerState(TypedDict):
  messages : Annotated[List[HumanMessage | AIMessage],"the messages in the conversation"]
  city: str
  interests: List[str]
  itinerary: str

API intergration

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
  temperature = 0,
  groq_api_key = "",
  model_name = "llama-3.3-70b-versatile"
)

In [ ]:
itinerary_prompt = ChatPromptTemplate.from_messages([
("system, You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
("human", "Create an itinerary for my day trip."),
])

Define agent function

In [ ]:
def input_city(state: PlannerState) -> PlannerState:
    """
    Collects the city name from the user and updates the state
    """
    print("Please enter the city you want to visit for your day trip:")
    user_message = input("Your Input: ")

    return {
        **state,
        "city": user_message,
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }


def input_interest(state: PlannerState) -> PlannerState:
    """
    Collects user interests and updates the state
    """
    print(
        f"Please enter your interests for the trip to {state['city']} "
        "(comma-separated):"
    )
    user_message = input("Your Input: ")

    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }


def create_itinerary(state: PlannerState) -> PlannerState:
    """
    Generates the travel itinerary using the LLM
    """
    print(
        f"\nCreating an itinerary for {state['city']} "
        f"based on interests: {', '.join(state['interests'])}"
    )

    response = llm.invoke(
        itinerary_prompt.format_messages(
            city=state["city"],
            interests=", ".join(state["interests"])
        )
    )

    print("\nFinal Itinerary:\n")
    print(response.content)

    return {
        **state,
        "itinerary": response.content,
        "messages": state["messages"] + [AIMessage(content=response.content)]
    }


create and compile the graph


In [ ]:
# ===================== CREATE THE WORKFLOW =====================

# Initialize the state graph with the PlannerState
workflow = StateGraph(PlannerState)

# ===================== ADD NODES =====================

# Each node represents one step in the agent workflow
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)

# ===================== SET ENTRY POINT =====================

# This is the first node that will run
workflow.set_entry_point("input_city")

# ===================== DEFINE EDGES (FLOW) =====================

# Define the order of execution
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)

# ===================== COMPILE THE GRAPH =====================

# Compile the workflow into an executable app
app = workflow.compile()


Define function that runs the graph

In [ ]:
def travel_planner(user_request: str):
    """
    Runs the LangGraph workflow for the travel planner
    """
    print(f"Initial Request: {user_request}\n")

    # Initial state passed into the graph
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",
        "interests": [],
        "itinerary": "",
    }

    # Execute the workflow (stream executes each node step-by-step)
    for _ in app.stream(state):
        pass


In [ ]:
!pip install gradio

UI

In [ ]:
pip install requests

In [ ]:
OPENWEATHER_API_KEY = ""
GOOGLE_MAPS_API_KEY = ""

In [ ]:
# ===================== IMPORTS =====================
import gradio as gr
import requests
from typing import TypedDict, Annotated, List

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


# ===================== API KEYS =====================
OPENWEATHER_API_KEY = ""
GOOGLE_MAPS_API_KEY = ""


# ===================== STATE DEFINITION =====================
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation messages"]
    start: str
    destination: str
    interests: List[str]
    itinerary: str


# ===================== LLM SETUP =====================
llm = ChatGroq(
    temperature=0,
    groq_api_key="",
    model="llama-3.3-70b-versatile"
)


# ===================== PROMPT =====================
itinerary_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful travel assistant. "
        "Create a one-day trip itinerary for {destination} "
        "based on interests: {interests}. Provide bullet points."
    ),
    ("human", "Create itinerary.")
])


# ===================== WEATHER FUNCTION =====================
def get_weather(city):
    url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    )

    data = requests.get(url).json()

    if "main" not in data:
        return "Weather data unavailable."

    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]

    return f"{temp}°C, {desc}"


# ===================== MAP EMBED =====================
def get_map_embed(start, destination):

    return f"""
    <iframe
        width="100%"
        height="400"
        style="border:0"
        loading="lazy"
        allowfullscreen
        src="https://www.google.com/maps/embed/v1/directions?key={GOOGLE_MAPS_API_KEY}&origin={start}&destination={destination}">
    </iframe>
    """


# ===================== ROUTE INFO =====================
def get_route_info(start, destination):

    url = (
        "https://maps.googleapis.com/maps/api/directions/json"
        f"?origin={start}&destination={destination}&key={GOOGLE_MAPS_API_KEY}"
    )

    data = requests.get(url).json()

    try:
        leg = data["routes"][0]["legs"][0]
        distance = leg["distance"]["text"]
        duration = leg["duration"]["text"]

        return f"Distance: {distance} | Travel Time: {duration}"

    except:
        return "Route info unavailable."


# ===================== ITINERARY =====================
def create_itinerary(destination, interests):

    response = llm.invoke(
        itinerary_prompt.format_messages(
            destination=destination,
            interests=interests
        )
    )

    return response.content


# ===================== MAIN FUNCTION =====================
def travel_planner(start, destination, interests):

    weather = get_weather(destination)
    route = get_route_info(start, destination)
    itinerary = create_itinerary(destination, interests)
    map_html = get_map_embed(start, destination)

    result = f"""
### 🌦️ Weather in {destination}
{weather}

### 🛣️ Route Information
{route}

### 🗺️ Itinerary
{itinerary}
"""

    return result, map_html


# ===================== GRADIO UI =====================
interface = gr.Interface(
    fn=travel_planner,

    inputs=[
        gr.Textbox(label="Start Location", placeholder="e.g., Hyderabad"),
        gr.Textbox(label="Destination City", placeholder="e.g., Nashik"),
        gr.Textbox(label="Interests", placeholder="temples, trekking, food"),
    ],

    outputs=[
        gr.Markdown(label="Trip Details"),
        gr.HTML(label="Map"),
    ],

    title="AI Travel Planner with Weather & Maps",
    description="Enter start location, destination, and interests."
)


# ===================== RUN =====================
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84ff12c30ca6bb2934.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
